In [1]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import utils

In [2]:
def load_data(path):
    df_summary = pd.read_csv("./medical/data_summary.csv")
    df_summary["filename"] = df_summary["filename"].apply(lambda x: x.split(".")[0])
    df_summary.head()

    FOLDER_PATH = path


    IMAGE_EXTENSIONS = ['.png', '.jpg', '.jpeg']
    data_list = []

    for filename in os.listdir(FOLDER_PATH):
        if filename.endswith('.npz'):
            file_stem = os.path.splitext(filename)[0]
            npz_path = os.path.join(FOLDER_PATH, filename)
            npz_data = np.load(npz_path, allow_pickle=True)
            id = file_stem.split("_")[-1]


            data_entry = {
                "filename": file_stem,
                "slo_fundus": npz_data["slo_fundus"],
            }

            data_list.append(data_entry)

    print(f"Loaded {len(data_list)} entries successfully.")
    df = pd.DataFrame(data_list)
    df = pd.merge(df, df_summary, on='filename', how='inner').drop(columns = ["use"])
    return df

df_train = load_data("./medical/Training")
df_test = load_data("./medical/Test")
df_val = load_data("./medical/Validation")

Loaded 7000 entries successfully.
Loaded 2000 entries successfully.
Loaded 1000 entries successfully.


# Analyse de la distribution 

In [17]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

def plot_distributions(df_train, df_test, df_val):
    df_train['age_rounded'] = df_train['age'].round()
    df_test['age_rounded'] = df_test['age'].round()
    df_val['age_rounded'] = df_val['age'].round()

    fig = make_subplots(
        rows=7, cols=3,
        subplot_titles=[
            "Race Distribution (Train)", "Race Distribution (Test)", "Race Distribution (Val)",
            "Gender Distribution (Train)", "Gender Distribution (Test)", "Gender Distribution (Val)",
            "Age Distribution (Train)", "Age Distribution (Test)", "Age Distribution (Val)",
            "Ethnicity Distribution (Train)", "Ethnicity Distribution (Test)", "Ethnicity Distribution (Val)",
            "Language Distribution (Train)", "Language Distribution (Test)", "Language Distribution (Val)",
            "Marital Status Distribution (Train)", "Marital Status Distribution (Test)", "Marital Status Distribution (Val)",
            "Healthy vs Sick (Train)", "Healthy vs Sick (Test)", "Healthy vs Sick (Val)"
        ],
        specs=[
            [{"type": "pie"}, {"type": "pie"}, {"type": "pie"}],
            [{"type": "pie"}, {"type": "pie"}, {"type": "pie"}],
            [{"type": "scatter"}, {"type": "scatter"}, {"type": "scatter"}],
            [{"type": "pie"}, {"type": "pie"}, {"type": "pie"}],
            [{"type": "pie"}, {"type": "pie"}, {"type": "pie"}],
            [{"type": "pie"}, {"type": "pie"}, {"type": "pie"}],
            [{"type": "pie"}, {"type": "pie"}, {"type": "pie"}]
        ]
    )

    for i, (df, title) in enumerate(zip([df_train, df_test, df_val], ["Train", "Test", "Val"])):
        race_counts = df['race'].value_counts()
        fig.add_trace(go.Pie(labels=race_counts.index, values=race_counts.values, name=f"Race {title}"), row=1, col=i + 1)

        gender_counts = df['gender'].value_counts()
        fig.add_trace(go.Pie(labels=gender_counts.index, values=gender_counts.values, name=f"Gender {title}"), row=2, col=i + 1)

        age_counts = df['age_rounded'].value_counts().sort_index()
        fig.add_trace(go.Scatter(x=age_counts.index, y=age_counts.values, mode='lines+markers', name=f"Age {title}"), row=3, col=i + 1)

        ethnicity_counts = df['ethnicity'].value_counts()
        fig.add_trace(go.Pie(labels=ethnicity_counts.index, values=ethnicity_counts.values, name=f"Ethnicity {title}"), row=4, col=i + 1)

        language_counts = df['language'].value_counts()
        fig.add_trace(go.Pie(labels=language_counts.index, values=language_counts.values, name=f"Language {title}"), row=5, col=i + 1)

        marital_status_counts = df['maritalstatus'].value_counts()
        fig.add_trace(go.Pie(labels=marital_status_counts.index, values=marital_status_counts.values, name=f"Marital Status {title}"), row=6, col=i + 1)

        glaucoma_counts = df['glaucoma'].value_counts()
        fig.add_trace(go.Pie(labels=glaucoma_counts.index, values=glaucoma_counts.values, name=f"Healthy vs Sick {title}"), row=7, col=i + 1)

    fig.update_layout(
        title="Comparison of Distributions Across Datasets",
        height=2100,  # Adjusted height for 7 rows
        width=1200,
        showlegend=False
    )

    fig.show()

plot_distributions(df_train, df_test, df_val)

In [4]:
df_combined = pd.concat([df_train, df_test, df_val], axis=0, ignore_index=True)
df_combined['dataset'] = ['Train'] * len(df_train) + ['Test'] * len(df_test) + ['Val'] * len(df_val)
df_combined.head()

,filename,slo_fundus,age,gender,race,ethnicity,language,maritalstatus,note,gpt4_summary,glaucoma,age_rounded,dataset
0,data_05935,"[[41, 37, 39, 41, 45, 40, 39, 37, 35, 42, 37, ...",52.80,male,white,non-hispanic,english,married or partnered,wq patient: called patient x2 on DATE_TIME to ...,The clinical note does not provide information...,yes,53.0,Train
1,data_03490,"[[46, 51, 46, 44, 48, 48, 44, 44, 48, 43, 41, ...",62.45,male,white,unknown,english,unknown,acute ischemia. 2. ÿunremarkable brain parench...,"Patient has acute ischemia, optic neuropathies...",yes,62.0,Train
2,data_00634,"[[6, 9, 4, 4, 5, 8, 9, 10, 9, 9, 8, 7, 8, 11, ...",71.98,female,asian,non-hispanic,english,married or partnered,"71 y.o. female f/u plateau PERSON, s/p lpi ou,...",71-year-old female with glaucoma suspect due t...,no,72.0,Train
3,data_00777,"[[23, 23, 25, 26, 28, 26, 27, 27, 24, 26, 25, ...",46.78,female,white,non-hispanic,english,married or partnered,3) would look into seeing a headache or interv...,The note discusses potential visits to a pain ...,yes,47.0,Train
4,data_00405,"[[20, 22, 22, 21, 22, 24, 22, 23, 22, 23, 27, ...",30.58,female,white,unknown,english,single,imp: physiologic cupping vs glaucoma suspect b...,Patient possibly has physiological cupping or ...,no,31.0,Train


In [ ]:
from aif360.sklearn.metrics import *


def get_group_metrics(
    y_true,
    y_pred=None,
    prot_attr=None,
    priv_group=1,
    pos_label=1,
    sample_weight=None,
):
    group_metrics = {}
    group_metrics["base_rate"] = base_rate(
        y_true=y_true, pos_label=pos_label, sample_weight=sample_weight
    )
    group_metrics["statistical_parity_difference"] = statistical_parity_difference(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
    )
    group_metrics["disparate_impact_ratio"] = disparate_impact_ratio(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
    )
    if not y_pred is None:
        group_metrics["equal_opportunity_difference"] = equal_opportunity_difference(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["average_odds_difference"] = average_odds_difference(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["conditional_demographic_disparity"] = conditional_demographic_disparity(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["smoothed_edf"] = smoothed_edf(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["df_bias_amplification"] = df_bias_amplification(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
    return group_metrics




ideal_values = {
    "statistical_parity_difference": 0.0,
    "disparate_impact_ratio": 1.0,
    "equal_opportunity_difference": 0.0,
    "average_odds_difference": 0.0,
    "conditional_demographic_disparity": 0.0,
    "smoothed_edf": 1.0,
    "df_bias_amplification": 0.0
}



def check_distance_to_ideal(metrics):
    dist = 0.0
    for metric in ideal_values.keys():
        dist += (ideal_values[metric]-metrics[metric])**2
    return np.sqrt(dist)

pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'
pip install 'aif360[OptimalTransport]'


In [19]:
from sklearn.preprocessing import LabelEncoder
from aif360.datasets import BinaryLabelDataset

df_test_numeric = df_test.drop(columns=['filename', 'slo_fundus', 'age', 'note', 'gpt4_summary'])

def convert_to_numerical_with_encodings(df):
    df_numeric = df.copy()
    encodings = {}
    for col in df_numeric.columns:
        if df_numeric[col].dtype == 'object':  # Check if the column is categorical
            le = LabelEncoder()
            df_numeric[col] = le.fit_transform(df_numeric[col])
            encodings[col] = dict(zip(le.classes_, le.transform(le.classes_)))  # Store the encoding
    return df_numeric, encodings


df_test_numeric, encodings = convert_to_numerical_with_encodings(df_test_numeric)


def create_binary_label_dataset(df, label_column, protected_attributes):
    return BinaryLabelDataset(
        df=df,
        label_names=[label_column],
        protected_attribute_names=protected_attributes
    )


label_column = 'glaucoma'  
protected_attributes = ['gender', 'race', 'ethnicity', 'language', 'maritalstatus']  


dataset = create_binary_label_dataset(df_test_numeric, label_column, protected_attributes)

In [10]:
encodings

{'gender': {'female': np.int64(0), 'male': np.int64(1)},
 'race': {'asian': np.int64(0), 'black': np.int64(1), 'white': np.int64(2)},
 'ethnicity': {'hispanic': np.int64(0),
  'non-hispanic': np.int64(1),
  'unknown': np.int64(2)},
 'language': {'english': np.int64(0),
  'other': np.int64(1),
  'spanish': np.int64(2),
  'unknown': np.int64(3)},
 'maritalstatus': {'divorced': np.int64(0),
  'legally separated': np.int64(1),
  'married or partnered': np.int64(2),
  'single': np.int64(3),
  'unknown': np.int64(4),
  'widowed': np.int64(5)},
 'glaucoma': {'no': np.int64(0), 'yes': np.int64(1)}}

In [20]:
priviliged_groups = [0,2,1,0,2]
protected_attributes = dataset.protected_attributes
y_true = dataset.labels


n_prot_attrs = protected_attributes.shape[1]
attribute_names = ['gender', 'race', 'ethnicity', 'language', 'maritalstatus']  

all_metrics = []

for i, attr_name in enumerate(attribute_names):
    prot_attr = protected_attributes[:, i]
    priv_group = priviliged_groups[i]
    
    try:
        metrics = get_group_metrics(
            y_true=y_true,
            y_pred=None,  
            prot_attr=prot_attr,
            priv_group=priv_group,
            pos_label=1
        )
        metrics['attribute'] = attr_name
        all_metrics.append(metrics)
    except Exception as e:
        print(f"[{attr_name}] Error computing metrics: {e}")


In [21]:
df_metrics = pd.DataFrame(all_metrics)
df_melted = df_metrics.melt(id_vars='attribute', var_name='metric', value_name='value')
df_melted = df_melted.dropna(subset=['value'])
fig = go.Figure()

for metric_name in df_melted['metric'].unique():
    if metric_name == 'attribute':
        continue
    df_metric = df_melted[df_melted['metric'] == metric_name]
    fig.add_trace(go.Bar(
        x=df_metric['attribute'],
        y=df_metric['value'],
        name=metric_name
    ))

fig.update_layout(
    barmode='group',
    title='Fairness Metrics by Protected Attribute',
    xaxis_title='Protected Attribute',
    yaxis_title='Metric Value',
    legend_title='Metric'
)

fig.show()


Loaded test set with predictions and prepared for fairness analysis

In [22]:
df_test_preds = pd.read_csv("./medical/df_test_with_preds.csv")
df_test_preds['filename'] = df_test_preds['filename'].apply(lambda x: x.split(".")[0])
df_test_preds = df_test_preds.drop(columns=['slo_fundus', 'age', 'note', 'gpt4_summary'])
protected_attributes = ['gender', 'race', 'ethnicity', 'language', 'maritalstatus'] 
df_test_preds_numeric, _ = convert_to_numerical_with_encodings(df_test_preds)

dataset_preds = create_binary_label_dataset(df_test_preds_numeric, label_column, protected_attributes)
protected_attributes_preds = dataset_preds.protected_attributes
y_true_preds = dataset_preds.labels
y_pred = df_test_preds_numeric['pred'].values

In [23]:
all_metrics_preds = []
for i, attr_name in enumerate(attribute_names):
    prot_attr = protected_attributes_preds[:, i]
    priv_group = priviliged_groups[i]
    try:
        metrics = get_group_metrics(
            y_true=y_true_preds,
            y_pred=y_pred,
            prot_attr=prot_attr,
            priv_group=priv_group,
            pos_label=1
        )
        metrics['attribute'] = attr_name
        all_metrics_preds.append(metrics)
    except Exception as e:
        print(f"[{attr_name}] Error computing metrics: {e}")

In [24]:
df_metrics_preds = pd.DataFrame(all_metrics_preds)
df_melted_preds = df_metrics_preds.melt(id_vars='attribute', var_name='metric', value_name='value')
df_melted_preds = df_melted_preds.dropna(subset=['value'])
fig_preds = go.Figure()
for metric_name in df_melted_preds['metric'].unique():
    if metric_name == 'attribute':
        continue
    df_metric = df_melted_preds[df_melted_preds['metric'] == metric_name]
    fig_preds.add_trace(go.Bar(
        x=df_metric['attribute'],
        y=df_metric['value'],
        name=metric_name
    ))
fig_preds.update_layout(
    barmode='group',
    title='Fairness Metrics by Protected Attribute (with Predictions)',
    xaxis_title='Protected Attribute',
    yaxis_title='Metric Value',
    legend_title='Metric',
    width=1200,
    height=700
)
fig_preds.show()

In [25]:
import plotly.colors

common_metrics = set(df_melted['metric']).intersection(set(df_melted_preds['metric']))
df_before = df_melted[df_melted['metric'].isin(common_metrics)]
df_after = df_melted_preds[df_melted_preds['metric'].isin(common_metrics)]

metric_names = list(common_metrics)
palette = plotly.colors.qualitative.Plotly  # or use another palette if you prefer
color_map = {metric: palette[i % len(palette)] for i, metric in enumerate(metric_names)}

fig_compare = make_subplots(
    rows=1, cols=2,
    subplot_titles=["Fairness Metrics Before Predictions", "Fairness Metrics After Predictions"],
    shared_yaxes=True
)

for metric in metric_names:
    df_b = df_before[df_before['metric'] == metric]
    df_a = df_after[df_after['metric'] == metric]
    color = color_map[metric]
    fig_compare.add_trace(
        go.Bar(
            x=df_b['attribute'],
            y=df_b['value'],
            name=metric,
            legendgroup=metric,
            showlegend=True,
            marker_color=color
        ),
        row=1, col=1
    )
    fig_compare.add_trace(
        go.Bar(
            x=df_a['attribute'],
            y=df_a['value'],
            name=metric,
            legendgroup=metric,
            showlegend=False,
            marker_color=color
        ),
        row=1, col=2
    )

fig_compare.update_layout(
    height=500,
    width=1100,
    barmode='group',
    title_text="Comparison of Fairness Metrics Before and After Predictions",
    legend_title="Metric"
)
fig_compare.show()